## Contexts

The Vcsn platform relies on a central concepts: "contexts".  They denote typing information about automata, rational expressions, etc.  This information is alike a function type: an input type (the _label_), and an output type (the _weight_).

Contexts are created by the `vcsn.context` function which takes a string as input.  This string follows the following syntax:

    <context> ::= <labelset> , <weightset>
    
i.e., a context name is composed of a labelset name, then a comma, then a weightset name.

## Labelsets

Different _LabelSets_ model multiple variations on _labels_,
members of a monoid:

* `letterset`

  Fully defined by an alphabet $A$, its labels being just
  letters.  It is simply denoted by $A$.  It corresponds to the usual
  definition of an NFA.

* `nullableset`

  Denoted by $A^?$, also defined by an alphabet $A$, its
  labels being either letters or the empty word.  This corresponds to what
  is often called $\varepsilon$-NFAs.
  
* `wordset`

  Denoted by $A^*$, also defined by an alphabet $A$, its labels
  being (possibly empty) words on this alphabet.

* `oneset`

  Denoted by $\{1\}$, containing a single label: 1, the empty word.

* `tupleset`

  Cartesian product of LabelSets, $L_1 \times \cdots \times
  L_n$. This type implements the concept of transducers with an arbitrary
  number of "tapes". The concept is developed more in-depth here: [Transducers](Transducers.ipynb).

## Weightsets

The _WeightSets_ define the semiring of the weights.  Builtin weights include:

* `b`

  The classical Booleans: $\langle \mathbb{B}, \vee, \wedge, \bot, \top \rangle$

* `z`

  The integers coded as `int`s: $\langle \mathbb{Z}, +, \times, 0, 1 \rangle$

* `q`

  The rationals, coded as pairs of `int`s: $\langle \mathbb{Q}, +, \times, 0, 1 \rangle$
  
* `qmp`

  The rationals, with support for multiprecision: $\langle \mathbb{Q}_\text{mp}, +, \times, 0, 1 \rangle$
  
* `r`

  The reals, coded as `double`s: $\langle \mathbb{R}, +, \times, 0, 1 \rangle$
  
* `zmin`

  The tropical semiring, coded as `int`s: $\langle \mathbb{Z} \cup \{\infty\}, \min, +, \infty, 0 \rangle$
  
* `rmin`

  The tropical semiring, coded as `floats`s: $\langle \mathbb{R} \cup \{\infty\}, \min, +, \infty, 0 \rangle$
  
* `log`

  The log semiring, coded as `double`s: $\langle \mathbb{R} \cup \{-\infty, +\infty\}, \oplus_\mathrm{log}, +, +\infty, 0 \rangle$ (where $\oplus_\mathrm{log}$ denotes $x, y \rightarrow - \mathrm{log}(\exp(-x) + \exp(-y))$.
* `f2`

  The field: $\langle \mathbb{F}_2, \oplus, \wedge, 0, 1 \rangle$ (where $\oplus$ denotes the "exclusive or").
  
* `tupleset`

  Cartesian product of WeightSets, $W_1 \times \cdots \times W_n$.

## Examples

The usual framework for automaton is to use letters as labels, and Booleans as weights:

In [1]:
import vcsn
vcsn.context('lal_char(abc), b')

:0: FutureWarning: IPython widgets are experimental and may change in the future.


<IPython.core.display.Javascript object>

letterset<char_letters(abc)>, b

If instead of a simple accepter that returns "yes" or "no", you want to compute an integer, work in $\mathbb{Z}$:

In [2]:
vcsn.context('lal_char(abc), z')

letterset<char_letters(abc)>, z

To use words on the usual alphabet as labels:

In [3]:
vcsn.context('law_char(a-z), z')

wordset<char_letters(abcdefghijklmnopqrstuvwxyz)>, z

To create a "classical" two-tape automaton:

In [4]:
vcsn.context('lat<lal_char(a-f), lal_char(A-F)>, b')

lat<letterset<char_letters(abcdef)>, letterset<char_letters(ABCDEF)>>, b

To compute a Boolean and an integer:

In [5]:
vcsn.context('lal_char(ab), lat<b, z>')

letterset<char_letters(ab)>, lat<b, z>

The interpretation of the following monster is left to the reader as an exercise:

In [6]:
vcsn.context('lan<lat<lal_char(ba),lat<lan<lal_char(vu)>,law_char(x-z)>>>,'
             +' lat<expressionset<lan<lat<lan_char(fe),lan_char(hg)>>, lat<r, q>>, lat<b, z>>')

nullableset<lat<letterset<char_letters(ab)>, lat<nullableset<letterset<char_letters(uv)>>, wordset<char_letters(xyz)>>>>, lat<expressionset<lat<nullableset<letterset<char_letters(ef)>>, nullableset<letterset<char_letters(gh)>>>, lat<r, q>>, lat<b, z>>